In [1]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef as Mcc
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.cross_validation import KFold

# Overview
This notebook uses a random forests to find what sequences can help differentiate between student groups
Each section has pseudo code outlining the steps of this analysis

Some resources:
* http://scikit-learn.org/stable/modules/ensemble.html#forest
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


# Here are some functions that might be useful

In [8]:
def fnrate(Y_true, Y_predict):
    '''Ouputs the true and false positive and negative rates of the classifier'''
    
    if Y_true.ndim != Y_predict.ndim: print "ERROR!!!"
    fn=0
    tp=0
    tn=0
    fp=0
    for i in range(0,len(Y_true)):
        if Y_true[i]==1:
           if Y_predict[i]==1:
              tp+=1
           if Y_predict[i]==0:
              fn+=1
        elif Y_true[i]==0:
           if Y_predict[i]==1:
              fp+=1
           if Y_predict[i] == 0:
              tn+=1
        else: print "Something is fishy"

    if len(Y_true)!= fn+fp+tp+tn: print "Oh no!"
    print "tp:",tp, "tn:", tn, "fp:", fp, "fn:", fn
    if fn+tp!= 0:
        print "Sensitivity ", float(tp)/float((fn+tp))
        print "Specificity ", float(tn)/float((fp+tn))
        return float(fn)/float((fn+tp))
    else: return 0.0

## Format the data
We need the data in a tabular format.
X = data = numpy array of 1 and 0 of size (# students by # sequences)
Y = student_groups = numpy array of 1 and 0 of size (# students by # groups)

In [10]:
# get sequences
# get students
# make data array
# make student group array
X = np.array([[0, 0], [1, 1]])
Y = np.array([0, 1])

## Run random forest
### First, we get our train and test sets

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.6)
print X_train.shape, Y_train.shape
print X_test.shape, Y_test.shape

### Second, we run the random forest on the training set

In [12]:
N_TREES = 50
MAX_TREE_DEPTH = 4
MIN_NUMBER_STUDENT_IN_LEAF = 2

#design the forest
clf = RandomForestClassifier(n_estimators=N_TREES, max_depth = MAX_TREE_DEPTH, min_samples_leaf = MIN_NUMBER_STUDENT_IN_LEAF, criterion="entropy", max_features="sqrt", bootstrap=True,verbose=1)

#train the forest
clf.fit(X_train,Y_train)

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=4, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

### Check success of random forest

In [13]:
mcc = Mcc(Y_test,clf.predict(X_test))
print "Results: ", mcc,
# print "False negative rate: ", fnrate(Y_test,clf.predict(X_test))

Results:  0.0 FN:  tp: 0 tn: 0 fp: 0 fn: 1
Sensitivity  0.0
Specificity 

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


ZeroDivisionError: float division by zero

### Now we extract the best features

## Cross validation
We want to determine the optimal number and depth of trees
We want to plot the score of the RF on the test data set by the number of trees
Then, we want to plot the score of the RF on the test data set by the depth of the tree

In [16]:
# for n_trees in []:
#     Split data using 5 fold cross validation
#     for each fold:
#         run RF
#         capture score